# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-e01ceea2ca-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Companies Workflow Series ("Business People in Germany") 

Consider the following exploratory information need:

> You are investigating important business people in Germany

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P17`      | country       | predicate |
| `wdt:P27`      | citizenship   | predicate |
| `wdt:P106`     | occupation   | predicate |
| `wd:Q183`      | Germany      | node      |
| `wd:Q2462658`  | manager      | node      |
| `wd:Q40966`    | Opel         | node      |
| `wd:Q56509715` | Michael Lohscheller | node |
| `wd:Q57479`    | Adam Opel    | node |



Also consider

```

?p wdt:P27 wd:Q183  . 
?p wdt:P106 wd:Q2462658  . 

```

is the BGP to retrieve all **german managers**

## Workload Goals

1. Identify the BGP for obtaining C.E.O., managers, directors, or founders of German companies

2. Identify the BGP to retrieve gender and profession of people and industry of companies

3. Are there german companies with C.E.O., managers, or founders that are not German?

4. Are there people related to multiple german companies?

5. Analyze the number of business people per role, type of company, and gender
 
   5.1 How many people for each role and gender are there in Germany?
   
   5.2 Are there companies with multiple german people having important roles?
   
   5.3 In which sectors are important german business people working?
   
   5.4 Are there german business people related to non german companies? How many?


1) First of all we retrieve the entities about the occupations we are interested in.

In [81]:
queryString = """
SELECT DISTINCT ?occupation ?oname
WHERE { 
    ?p wdt:P27 wd:Q183  . 
    ?p wdt:P106 ?occupation  .
    ?occupation <http://schema.org/name> ?oname
    FILTER REGEX(?oname, '^manager$|^director$|^founder$|Chief Executive Officer|creator', 'i')
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('occupation', 'http://www.wikidata.org/entity/Q3455803'), ('oname', 'director')]
[('occupation', 'http://www.wikidata.org/entity/Q2462658'), ('oname', 'manager')]
[('occupation', 'http://www.wikidata.org/entity/Q1162163'), ('oname', 'director')]
[('occupation', 'http://www.wikidata.org/entity/Q484876'), ('oname', 'chief executive officer')]
[('occupation', 'http://www.wikidata.org/entity/Q2500638'), ('oname', 'creator')]
[('occupation', 'http://www.wikidata.org/entity/Q721834'), ('oname', 'manager')]
[('occupation', 'http://www.wikidata.org/entity/Q6746710'), ('oname', 'manager')]
[('occupation', 'http://www.wikidata.org/entity/Q19435686'), ('oname', 'founder')]


8

There are some duplicates but we alredy know that Q2462658 is the correct one for 'manager'.

In [109]:
queryString = """
SELECT DISTINCT ?p ?pname ?occupationName
WHERE { 
    ?p wdt:P27 wd:Q183  . 
    ?p wdt:P106 ?occupation .
    ?p <http://schema.org/name> ?pname .
    ?occupation <http://schema.org/name> ?occupationName
    FILTER(?occupation = wd:Q3455803 || ?occupation = wd:Q2462658 || ?occupation = wd:Q1162163 || ?occupation = wd:Q484876 || ?occupation = wd:Q19435686 || ?occupation = wd:Q2500638)
}
LIMIT 30
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q1745683'), ('pname', 'Klaus Schrenk'), ('occupationName', 'director')]
[('p', 'http://www.wikidata.org/entity/Q57275'), ('pname', 'Ermanno Wolf-Ferrari'), ('occupationName', 'director')]
[('p', 'http://www.wikidata.org/entity/Q2589322'), ('pname', 'Wolfgang Brunecker'), ('occupationName', 'director')]
[('p', 'http://www.wikidata.org/entity/Q23789299'), ('pname', 'Ulrike Putz'), ('occupationName', 'director')]
[('p', 'http://www.wikidata.org/entity/Q61967768'), ('pname', 'Andrea Ammon'), ('occupationName', 'director')]
[('p', 'http://www.wikidata.org/entity/Q18602666'), ('pname', 'Walter Hofer'), ('occupationName', 'director')]
[('p', 'http://www.wikidata.org/entity/Q48689370'), ('pname', 'Wolfram Horstmann'), ('occupationName', 'director')]
[('p', 'http://www.wikidata.org/entity/Q64169746'), ('pname', 'Hermann Poppe-Marquard'), ('occupationName', 'director')]
[('p', 'http://www.wikidata.org/entity/Q20753171'), ('pname', 'Dieter Mitrenga')

30

2) We need to find the predicate for the industry companies.

In [117]:
queryString = """
SELECT DISTINCT ?p ?pname 
WHERE { 
    ?a wdt:P27 wd:Q183  . 
    ?a wdt:P106 ?occupation .
    ?a ?p ?o .
    ?p <http://schema.org/name> ?pname
    FILTER REGEX(?pname, 'work|work for|works for|company|companies|sector', 'i')
    }
    
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1441'), ('pname', 'present in work')]
[('p', 'http://www.wikidata.org/prop/direct/P2031'), ('pname', 'work period (start)')]
[('p', 'http://www.wikidata.org/prop/direct/P937'), ('pname', 'work location')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pname', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P6379'), ('pname', 'has works in the collection')]
[('p', 'http://www.wikidata.org/prop/direct/P1982'), ('pname', 'Anime News Network person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P800'), ('pname', 'notable work')]
[('p', 'http://www.wikidata.org/prop/direct/P5297'), ('pname', 'Companies House officer ID')]
[('p', 'http://www.wikidata.org/prop/direct/P4768'), ('pname', 'cinematografo.it name or company ID')]
[('p', 'http://www.wikidata.org/prop/direct/P6551'), ('pname', 'Physics History Network ID')]
[('p', 'http://www.wikidata.org/prop/direct/P953'), ('pname', 'full work available at URL')]
[('p', '

31

Some interesting predicates are 'field of work' (P101), 'notable work' (P800) and 'production company' (P272).
Now we need to find the predicate about gender.

In [26]:
queryString = """
SELECT DISTINCT ?p ?pname 
WHERE { 
    ?a wdt:P27 wd:Q183  . 
    ?a wdt:P106 ?occupation .
    ?a ?p ?o .
    ?p <http://schema.org/name> ?pname
    FILTER REGEX(?pname, 'gender', 'i')
    }
    
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P21'), ('pname', 'sex or gender')]


1

In [30]:
queryString = """
SELECT DISTINCT ?aname ?occupationName ?gendername ?fieldname
WHERE { 
    ?a wdt:P27 wd:Q183  . 
    ?a wdt:P106 ?occupation .
    ?a wdt:P21 ?gender .
    ?a wdt:P101 ?field .
    ?a <http://schema.org/name> ?aname .
    ?occupation <http://schema.org/name> ?occupationName .
    ?gender <http://schema.org/name> ?gendername .
    ?field <http://schema.org/name> ?fieldname .
}
    
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('aname', 'Erich Bulitta'), ('occupationName', 'non-fiction writer'), ('gendername', 'male'), ('fieldname', 'culture of remembrance')]
[('aname', 'Erich Bulitta'), ('occupationName', 'writer'), ('gendername', 'male'), ('fieldname', 'culture of remembrance')]
[('aname', 'Erich Bulitta'), ('occupationName', 'teacher'), ('gendername', 'male'), ('fieldname', 'culture of remembrance')]
[('aname', 'Hildegard Bulitta'), ('occupationName', 'non-fiction writer'), ('gendername', 'female'), ('fieldname', 'culture of remembrance')]
[('aname', 'Hildegard Bulitta'), ('occupationName', 'writer'), ('gendername', 'female'), ('fieldname', 'culture of remembrance')]
[('aname', 'Hildegard Bulitta'), ('occupationName', 'teacher'), ('gendername', 'female'), ('fieldname', 'culture of remembrance')]
[('aname', 'Max Ring'), ('occupationName', 'journalist'), ('gendername', 'male'), ('fieldname', 'medicine')]
[('aname', 'Max Ring'), ('occupationName', 'writer'), ('gendername', 'male'), ('fieldname', 'me

50

Let's investigate the node Opel.

In [52]:
queryString = """
SELECT DISTINCT ?pname ?p
WHERE { 
    wd:Q40966 ?p ?o .
    ?p <http://schema.org/name> ?pname
    FILTER REGEX(?pname, 'manager|director|founder|Chief Executive Officer', 'i')
}
    
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('pname', 'director / manager'), ('p', 'http://www.wikidata.org/prop/direct/P1037')]
[('pname', 'chief executive officer'), ('p', 'http://www.wikidata.org/prop/direct/P169')]


2

In [40]:
queryString = """
SELECT DISTINCT ?oname ?o
WHERE { 
    wd:Q40966 wdt:P31/wdt:P279* ?o .
    ?o <http://schema.org/name> ?oname
    FILTER REGEX(?oname, 'company', 'i')
    
}
    
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('oname', 'company'), ('o', 'http://www.wikidata.org/entity/Q783794')]
[('oname', 'commercial company'), ('o', 'http://www.wikidata.org/entity/Q567521')]
[('oname', 'private company'), ('o', 'http://www.wikidata.org/entity/Q5621421')]


3

In [48]:
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 
    wd:Q40966 ?p ?o .
    ?p <http://schema.org/name> ?pname
    FILTER REGEX(?pname, 'country', 'i')
    
}
    
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]


1

In [95]:
queryString = """
SELECT DISTINCT ?companyName ?nationality 
WHERE {
    ?company wdt:P31/wdt:P279* wd:Q783794 .
    ?company wdt:P17 wd:Q183 .
    ?company <http://schema.org/name> ?companyName .
    ?company wdt:P1037 ?dirman .
    ?dirman wdt:P27 ?citizenship .
    ?citizenship <http://schema.org/name> ?nationality 
    FILTER(?citizenship != wd:Q183)
    
}
    
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('companyName', 'Leonine Holding'), ('nationality', 'Austria')]
[('companyName', 'Springer'), ('nationality', 'Kingdom of the Netherlands')]
[('companyName', 'Direct Group'), ('nationality', 'Spain')]
[('companyName', 'Federal Agency for Civic Education'), ('nationality', 'German Democratic Republic')]
[('companyName', 'Schweigersche Buchhandlung'), ('nationality', 'German Empire')]
[('companyName', 'Universum Film AG'), ('nationality', 'German Reich')]
[('companyName', 'Pantera Film'), ('nationality', 'France')]
[('companyName', 'Rowboat Film- und Fernsehproduktion'), ('nationality', 'United States of America')]
[('companyName', 'Agfa-Gevaert'), ('nationality', 'France')]
[('companyName', 'Deutsch-Sorbisches Volkstheater'), ('nationality', 'German Democratic Republic')]
[('companyName', 'Kiel Institute for the World Economy'), ('nationality', 'United States of America')]
[('companyName', 'Max Planck Institute for the History of Science'), ('nationality', 'United States of Ame

18

As we can see there are many companies that have a director or a manager whitout a German citizenship.

4) Here we have director, managers or C.E.O. that are related to more than one company.

In [105]:
queryString = """
SELECT DISTINCT COUNT(?companyName) as ?howmany ?dirmanName ?ceoname
WHERE {
    ?company wdt:P31/wdt:P279* wd:Q783794 .
    ?company wdt:P17 wd:Q183 .
    ?company <http://schema.org/name> ?companyName .
    ?company wdt:P1037 ?dirman .
    ?dirman <http://schema.org/name> ?dirmanName
    OPTIONAL{
    ?company wdt:P169 ?ceo .
    ?ceo <http://schema.org/name> ?ceoname
    }
}
GROUP BY ?dirmanName ?ceoname
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('howmany', '4'), ('dirmanName', 'Herbert Diess'), ('ceoname', 'Herbert Diess')]
[('howmany', '3'), ('dirmanName', 'Christiane Ruff')]
[('howmany', '4'), ('dirmanName', 'Derk Haank')]
[('howmany', '2'), ('dirmanName', 'Gerhard Hahn')]
[('howmany', '4'), ('dirmanName', 'Oliver Bäte')]
[('howmany', '4'), ('dirmanName', 'Barbara Scheuch-Vötterle')]
[('howmany', '4'), ('dirmanName', 'Matthias Zachert')]
[('howmany', '3'), ('dirmanName', 'Romuald Karmakar')]
[('howmany', '6'), ('dirmanName', 'Rice Powell')]
[('howmany', '6'), ('dirmanName', 'Andreas Kennecke')]
[('howmany', '2'), ('dirmanName', 'Karsten Mühlenfeld')]
[('howmany', '2'), ('dirmanName', 'Stephan Stracke')]
[('howmany', '3'), ('dirmanName', 'Sam Davis')]
[('howmany', '2'), ('dirmanName', 'Dirk Stenkamp')]
[('howmany', '2'), ('dirmanName', 'Michael Kirchschlager')]
[('howmany', '4'), ('dirmanName', 'Klaus Servene')]
[('howmany', '2'), ('dirmanName', 'Maria Leptin')]
[('howmany', '2'), ('dirmanName', 'Lisa Davis')]
[('ho

20

5.1) WE retrieve German people grouped by role and gender.

In [108]:
queryString = """
SELECT DISTINCT COUNT(?p) as ?howmany ?occname ?gendername
WHERE {
    ?p wdt:P27 wd:Q183  .
    ?p wdt:P106 ?occupation .
    ?p wdt:P21 ?gender .
    ?occupation <http://schema.org/name> ?occname .
    ?gender <http://schema.org/name> ?gendername
    
}
GROUP BY ?occname ?gendername
ORDER BY DESC (COUNT(?p))
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('howmany', '31742'), ('occname', 'university teacher'), ('gendername', 'male')]
[('howmany', '28976'), ('occname', 'politician'), ('gendername', 'male')]
[('howmany', '16236'), ('occname', 'writer'), ('gendername', 'male')]
[('howmany', '13852'), ('occname', 'association football player'), ('gendername', 'female')]
[('howmany', '11097'), ('occname', 'association football player'), ('gendername', 'male')]
[('howmany', '8851'), ('occname', 'painter'), ('gendername', 'male')]
[('howmany', '7414'), ('occname', 'author'), ('gendername', 'male')]
[('howmany', '6488'), ('occname', 'journalist'), ('gendername', 'male')]
[('howmany', '6166'), ('occname', 'jurist'), ('gendername', 'male')]
[('howmany', '5700'), ('occname', 'theologian'), ('gendername', 'male')]
[('howmany', '4763'), ('occname', 'writer'), ('gendername', 'female')]
[('howmany', '4634'), ('occname', 'politician'), ('gendername', 'female')]
[('howmany', '4563'), ('occname', 'composer'), ('gendername', 'male')]
[('howmany'

20

5.2) Let's investigate the predicate we need to use.

In [110]:
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
    wd:Q56509715 ?p ?o .
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    
}

LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pname', 'place of birth'), ('o', 'http://www.wikidata.org/entity/Q3917'), ('oname', 'Bocholt')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('pname', 'country of citizenship'), ('o', 'http://www.wikidata.org/entity/Q183'), ('oname', 'Germany')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('pname', 'languages spoken, written or signed'), ('o', 'http://www.wikidata.org/entity/Q188'), ('oname', 'German')]
[('p', 'http://www.wikidata.org/prop/direct/P735'), ('pname', 'given name'), ('o', 'http://www.wikidata.org/entity/Q4927524'), ('oname', 'Michael')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q5'), ('oname', 'human')]
[('p', 'http://www.wikidata.org/prop/direct/P21'), ('pname', 'sex or gender'), ('o', 'http://www.wikidata.org/entity/Q6581097'), ('oname', 'male')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('

8

In [111]:
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
    ?o ?p wd:Q56509715 .
    ?p <http://schema.org/name> ?pname .
    ?o <http://schema.org/name> ?oname .
    
}

LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1037'), ('pname', 'director / manager'), ('o', 'http://www.wikidata.org/entity/Q40966'), ('oname', 'Opel')]


1

In [116]:
queryString = """
SELECT DISTINCT ?companyname ?dirmanName ?occupationName
WHERE {
    ?company wdt:P31/wdt:P279* wd:Q783794 .
    ?company wdt:P1037 ?dirman .
    ?dirman wdt:P27 wd:Q183 .
    ?dirman wdt:P106 ?occupation .
    ?company <http://schema.org/name> ?companyname .
    ?dirman <http://schema.org/name> ?dirmanName .
    ?occupation <http://schema.org/name> ?occupationName
    FILTER REGEX(?occupationName, '^manager$|^director$|^founder$|Chief Executive Officer|creator', 'i')
}
ORDER BY (?companyname)
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('companyname', 'Berliner Verkehrsbetriebe'), ('dirmanName', 'Sigrid Nikutta'), ('occupationName', 'manager')]
[('companyname', 'Bertelsmann'), ('dirmanName', 'Mark Wössner'), ('occupationName', 'manager')]
[('companyname', 'Bertelsmann'), ('dirmanName', 'Gunter Thielen'), ('occupationName', 'manager')]
[('companyname', 'Bertelsmann'), ('dirmanName', 'Manfred Fischer'), ('occupationName', 'manager')]
[('companyname', 'DB Netz'), ('dirmanName', 'Frank Sennhenn'), ('occupationName', 'manager')]
[('companyname', 'Deutsche Bahn'), ('dirmanName', 'Richard Lutz'), ('occupationName', 'manager')]
[('companyname', 'Deutsche Post AG'), ('dirmanName', 'Frank Appel'), ('occupationName', 'chief executive officer')]
[('companyname', 'Deutsche Reichsbahn (GDR)'), ('dirmanName', 'Herbert Keddi'), ('occupationName', 'manager')]
[('companyname', 'Deutsche Reichsbahn (GDR)'), ('dirmanName', 'Otto Arndt'), ('occupationName', 'manager')]
[('companyname', 'Deutsche Reichsbahn (GDR)'), ('dirmanName'

20

In [118]:
queryString = """
SELECT DISTINCT ?peoplename ?occupationName ?fieldname
WHERE {
    ?people wdt:P106 ?occupation .
    ?people <http://schema.org/name> ?peoplename .
    ?occupation <http://schema.org/name> ?occupationName .
    ?people wdt:P101 ?field .
    ?field <http://schema.org/name> ?fieldname
    FILTER REGEX(?occupationName, '^manager$|^director$|^founder$|Chief Executive Officer|creator', 'i')
    
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('peoplename', 'Michel Clement'), ('occupationName', 'manager'), ('fieldname', 'media management')]
[('peoplename', 'Paul Weis'), ('occupationName', 'director'), ('fieldname', 'Refugee law')]
[('peoplename', 'Dafydd Johnston'), ('occupationName', 'director'), ('fieldname', 'Welsh poetry')]
[('peoplename', 'Fiona Watt'), ('occupationName', 'chief executive officer'), ('fieldname', 'regenerative medicine')]
[('peoplename', 'Fiona Watt'), ('occupationName', 'director'), ('fieldname', 'regenerative medicine')]
[('peoplename', 'Lady Amanda Ellingworth'), ('occupationName', 'director'), ('fieldname', 'home care')]
[('peoplename', 'Ed Burrows'), ('occupationName', 'manager'), ('fieldname', 'public broadcasting')]
[('peoplename', 'Tony Robbins'), ('occupationName', 'manager'), ('fieldname', 'neuro-linguistic programming')]
[('peoplename', 'Bill Gosden'), ('occupationName', 'director'), ('fieldname', 'film studies')]
[('peoplename', 'Anna H. Spitz'), ('occupationName', 'director'), ('f

20

5.4)Here we have all the non German companies with business people with a german citizenship.

In [133]:
queryString = """
SELECT DISTINCT ?companyname ?countryname ?dirmanName
WHERE {
    ?company wdt:P31/wdt:P279* wd:Q783794 .
    ?company wdt:P1037/wdt:P169* ?dirman .
    ?dirman wdt:P27 wd:Q183 .
    ?company wdt:P17 ?country.
    ?company <http://schema.org/name> ?companyname .
    ?dirman <http://schema.org/name> ?dirmanName .
    ?country <http://schema.org/name> ?countryname
    FILTER (?country != wd:Q183 && ?country != wd:Q16957)
    
    
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('companyname', 'Verlag Der Syndikalist'), ('countryname', 'Weimar Republic'), ('dirmanName', 'Fritz Kater')]
[('companyname', 'Atrium Verlag'), ('countryname', 'Switzerland'), ('dirmanName', 'Silke Weitendorf')]
[('companyname', 'Vodafone'), ('countryname', 'United Kingdom'), ('dirmanName', 'Gerard Kleisterlee')]
[('companyname', 'Vodafone'), ('countryname', 'United Kingdom'), ('dirmanName', 'Mathias Döpfner')]
[('companyname', 'Allgemeiner Rabbiner-Verband in Deutschland'), ('countryname', 'German Empire'), ('dirmanName', 'Leo Baeck')]
[('companyname', 'Learned Estonian Society'), ('countryname', 'Estonia'), ('dirmanName', 'Leo Meyer')]
[('companyname', 'Learned Estonian Society'), ('countryname', 'Estonia'), ('dirmanName', 'Walter Anderson')]
[('companyname', 'Learned Estonian Society'), ('countryname', 'Estonia'), ('dirmanName', 'Carl Schirren')]
[('companyname', 'OGX'), ('countryname', 'Brazil'), ('dirmanName', 'Eike Batista')]
[('companyname', 'UBS'), ('countryname', 'Sw

20